In [ ]:
# Location Analysis
# Working with the module Folium

# In order to see the map visualization, have to run the analysis using Jupyter Notebook

# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import time
import sys

In [ ]:
import sys
import subprocess

def installPackage(package):
    p = subprocess.run([sys.executable, "-m", "pip", "install", "-U", package], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print(p.stdout.decode())

requirements = ["folium", "leafmap", "ipyleaflet"]
for requirement in requirements:
    installPackage(requirement)

In [ ]:
# Bring in the additional libraries
import folium
import leafmap.foliumap as leafmap

In [ ]:
# Bring in the Irish towns dataset
df = pd.read_csv('input_data.csv')

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
# Review the columns within the DataFrame
df.columns

In [ ]:
# Colour options from folium
fol_colours = ['pink', 'darkred', 'gray', 'red', 'lightblue', 'lightgray', 'blue', 'white', 'darkgreen', 'black', 'darkpurple', 'orange', 'green', 'cadetblue', 'beige', 'darkblue', 'lightgreen', 'lightred', 'purple']
len(fol_colours)
fol_colours[:14]

In [ ]:
# Combine two lists for icon
facility_cat = [i for i in df['Facility Category'].unique()]
purpose_colour = dict(zip(facility_cat, fol_colours[:len(facility_cat)]))

In [ ]:
purpose_colour

In [ ]:
# Create the RGB colour mapping table for the leafmap library
import random as r
num_colors = 14
hex_colors = ['#' + ''.join([r.choice('0123456789ABCDEF') for _ in range(6)]) for _ in range(num_colors)]
print(hex_colors)

In [ ]:
# Hex colour list
hex_colours = ['#04F9E8', '#C44585', '#5722A1', '#D8F296', '#022EAB', '#E900C5', '#A7B3E3', '#64AD52', '#0B5348', '#5E7FDB', '#5CC8FF', '#D9E136', '#2C7786', '#D6DE0D']
purpose_colour1 = dict(zip(facility_cat, hex_colours[:len(facility_cat)]))

In [ ]:
purpose_colour1

In [ ]:
# Grouped by facility type
grouped = df.groupby('Facility Category')

In [ ]:
# Build the map
m1 = folium.Map(location=[df.Latitude.mean(), df.Longitude.mean()]
               ,zoom_start=3
               ,control_scale=True)

# Loop through each row in the dataframe
for i,row in df.iterrows():
    # Setup the content of group
    iframe = folium.IFrame(f'Owner Name: {str(row["Owner Name"])} \n Asset Name: {str(row["Asset Name"])} \n Facility: {str(row["Facility Category"])}'
                          ,width=300
                          ,height=200)
    # Initialise the popup using the iframe
    popup = folium.Popup(iframe, max_width=650)

    # Icon colour
    try:
        icon_colour = purpose_colour[row['Facility Category']]
    except:
        icon_colour = 'gray'
    
    # Add each row to the map
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']]
                        ,radius=2
                        ,popup=popup
                        ,fill_color=icon_colour
                        ,color=icon_colour
                 ).add_to(m1)

# Display map
m1

In [ ]:
# Create map using leafmap
m2 = leafmap.Map(location=[df.Latitude.mean(), df.Longitude.mean()]
               ,zoom_start=3)
    
# Loop through each row in the dataframe
for i,row in df.iterrows():

    # Icon colour
    try:
        icon_colour = purpose_colour1[row['Facility Category']]
    except:
        icon_colour = 'gray'
    
    # Add each row to the map
    m2.add_circle_markers_from_xy(
        data=df
        ,x='Longitude'
        ,y='Latitude'
        ,radius=2
        ,popup=[str(row["Owner Name"]), str(row["Asset Name"]), str(row["Facility Category"])]
        ,fill_color=icon_colour
        ,color=icon_colour
    )

m2.add_legend(legend_dict=purpose_colour1)

# Display map
m2